# Low-Level Design (LLD) of a Car Rental System

### 1. Requirements

1. **Renting a Car**:
    - The system should allow users to rent a car based on availability, vehicle type, rental duration, and other rental attributes

2. **Returning a Car**:
    - The system should allow users to return the rented car, calculate rental fees, and mark the car as available again.

3. **Car Availability:**:
    - Track the availability of cars in the system based on their current rental status.

4. **Car Reservation**:
    - Allow users to reserve a car for a specific time and date.

5. **Rental Fees**:
    - The rental fee should be calculated based on the car's rental duration, type.

6. **Customer Management**:
    - The system should allow the creation of customer profiles with personal details, rental history, etc

7. **Multiple Car Types**:
    - Support for different types of cars (e.g., sedan, SUV, convertible, etc.) and unique rates for each type.

---

### 2. Constraints

1. The system should handle multiple car types (e.g., sedan, SUV, etc.) and calculate different rental rates accordingly
2. The system should check car availability in real-time and prevent double bookings
3. There should be a maximum rental duration for each car.
4. The system must track all rented cars and maintain rental history.

---

### 3. Identify Entities

1. **Car**:
    - Represents a car with attributes such as ID, type (sedan, SUV, etc.), availability status, and rental details.

2. **Customer**:
    - Represents a customer who rents the car, with personal details and rental history.

3. **RentalAgreement**:
    - Represents a rental agreement between the customer and the rental company. It holds the details of the rental duration, rental fee, and the car.

4. **RentalFeeCalculator**:
    - Responsible for calculating the rental fee based on car type, rental duration, and any additional services

5. **Reservation**:
    - Represents a reservation made by the customer for a car. It includes details like reservation time, car, and customer.

6. **CarRentalSystem**:
    - Manages the car rental process, including adding cars, renting cars, handling reservations, and calculating rental fees

### 4. Class Design

#### 4.1. Car Class

In [5]:
class Car:
    def __init__(self, car_id, car_type, rental_price):
        self.car_id = car_id
        self.car_type = car_type
        self.is_available = True
        self.rental_price = rental_price

    def mark_unavailable(self):
        self.is_available = False

    def mark_available(self):
        self.is_available = True


#### 4.2. Customer Class

In [7]:
class Customer:
    def __init__(self, customer_id, name, address):
        self.customer_id = customer_id
        self.name = name
        self.address = address
        self.rental_history = []

    def add_rental_history(self, rental_agreement):
        self.rental_history.append(rental_agreement)


#### 4.3. RentalAgreement Class

In [9]:
from datetime import datetime

class RentalAgreement:
    def __init__(self, car, customer, start_time, end_time):
        self.car = car
        self.customer = customer
        self.start_time = start_time
        self.end_time = end_time
        self.total_fee = 0

    def calculate_total_fee(self, fee_calculator):
        self.total_fee = fee_calculator.calculate_fee(self.car, self.start_time, self.end_time)


#### 4.4. RentalFeeCalculator Class

In [11]:
class RentalFeeCalculator:
    def calculate_fee(self, car, start_time, end_time):
        duration = (end_time - start_time).days  # rental duration in days
        rate = self.get_rate(car.car_type)
        fee = duration * rate
        return fee

    def get_rate(self, car_type):
        rates = {
            "sedan": 30,   # per day
            "SUV": 50,
            "convertible": 70,
        }
        return rates.get(car_type, 30)  # default rate for unknown types


#### 4.5. Reservation Class

In [13]:
class Reservation:
    def __init__(self, reservation_id, car, customer, reservation_date):
        self.reservation_id = reservation_id
        self.car = car
        self.customer = customer
        self.reservation_date = reservation_date


#### 4.6. CarRentalSystem Class

In [15]:
class CarRentalSystem:
    def __init__(self):
        self.cars = []  # List of all cars
        self.customers = []  # List of all customers
        self.rentals = []  # List of active rentals

    def add_car(self, car):
        """Add a car to the system."""
        self.cars.append(car)

    def add_customer(self, customer):
        """Add a customer to the system."""
        self.customers.append(customer)

    def check_availability(self, car_id):
        """Check if a car is available for rent."""
        for car in self.cars:
            if car.car_id == car_id:
                if car.is_available:
                    return True
                else:
                    raise CarNotAvailableException(f"Car {car_id} is already rented")

        raise CarNotFoundException(f"Car {car_id} not found")

    def rent_car(self, customer, car_id, start_time, end_time):
        """Rent a car to a customer."""
        # Check if customer exists
        if customer not in self.customers:
            raise CustomerNotFoundException(f"Customer {customer.customer_id} not found")
        
        # Check if car is available
        self.check_availability(car_id)
        
        # Validate rental period
        if start_time >= end_time:
            raise InvalidRentalPeriodException("Start time must be before end time")
        
        # Rent the car
        car = next(car for car in self.cars if car.car_id == car_id)
        car.mark_unavailable()
        
        rental_agreement = RentalAgreement(car, customer, start_time, end_time)
        rental_agreement.calculate_total_fee(RentalFeeCalculator())
        
        customer.add_rental_history(rental_agreement)
        self.rentals.append(rental_agreement)  # Track active rental
        return rental_agreement

    def return_car(self, rental_agreement):
        """Return a rented car."""
        # Check if car is rented
        if rental_agreement.car not in [rental.car for rental in self.rentals]:
            raise InvalidReturnCarException(f"Car {rental_agreement.car.car_id} was not rented or already returned")
        
        # Mark the car as available
        rental_agreement.car.mark_available()
        self.rentals.remove(rental_agreement)  # Remove from active rentals
        
        return rental_agreement.total_fee


### 5. Exception Handling

In [17]:
# Base class for custom exceptions
class CarRentalException(Exception):
    """Base class for all custom exceptions."""
    pass


class CarNotAvailableException(CarRentalException):
    """Raised when the requested car is not available."""
    def __init__(self, message="Car is not available for rental"):
        self.message = message
        super().__init__(self.message)


class CarNotFoundException(CarRentalException):
    """Raised when a car with the specified ID is not found."""
    def __init__(self, message="Car not found in the system"):
        self.message = message
        super().__init__(self.message)


class CustomerNotFoundException(CarRentalException):
    """Raised when a customer with the specified ID is not found."""
    def __init__(self, message="Customer not found in the system"):
        self.message = message
        super().__init__(self.message)


class InvalidReservationException(CarRentalException):
    """Raised when a customer attempts to rent a car without a valid reservation."""
    def __init__(self, message="Reservation is required to rent a car"):
        self.message = message
        super().__init__(self.message)


class InvalidRentalPeriodException(CarRentalException):
    """Raised when the rental period is invalid."""
    def __init__(self, message="Invalid rental period. Please specify a valid duration"):
        self.message = message
        super().__init__(self.message)


class InvalidReturnCarException(CarRentalException):
    """Raised when an attempt is made to return a car that wasn't rented."""
    def __init__(self, message="This car was not rented or is already returned"):
        self.message = message
        super().__init__(self.message)

### 6. Implementation

In [19]:
# Creating car rental system
car_rental_system = CarRentalSystem()

# Adding cars
car1 = Car("C001", "sedan", 30)
car2 = Car("C002", "SUV", 50)
car_rental_system.add_car(car1)
car_rental_system.add_car(car2)

# Creating customers
customer1 = Customer("CUS001", "Alice", "123 Main St")
customer2 = Customer("CUS002", "Bob", "456 Elm St")
car_rental_system.add_customer(customer1)
car_rental_system.add_customer(customer2)

# Renting a car
from datetime import datetime, timedelta

start_time = datetime.now()
end_time = start_time + timedelta(days=5)

rental_agreement1 = car_rental_system.rent_car(customer1, "C001", start_time, end_time)
print(f"Rental fee for car {rental_agreement1.car.car_id} is ${rental_agreement1.total_fee}")

# Returning the car
fee = car_rental_system.return_car(rental_agreement1)
print(f"Final fee for car {rental_agreement1.car.car_id} is ${fee}")

# check availability
try:
    car3 = Car("C003", "SUV", 100)
    car_rental_system.check_availability(car3.car_id)
except CarNotFoundException as e:
    print(e)


# Customer not found
try:
    customer3 = Customer("CUS002", "Alice", "123 Main St")
    car_rental_system.rent_car(customer3, "C001", start_time, end_time)
except CustomerNotFoundException as e:
    print(e)


Rental fee for car C001 is $150
Final fee for car C001 is $150
Car C003 not found
Customer CUS002 not found
